# Synthetic Tabular Data Generation by NeMo GPT

Big data, new algorithms and fast computers are the 3 main factors that make the modern AI revolution possible. Data poses a big challenge for enterprises for various reasons: difficulty  of data labeling, strong data governance, limited data availability etc. Synthetic data generation is proposed as a solution to the data problem as it directly addresses the data challenges. Two popular generative models like  Variational Auto-Encoder [VAE](https://en.wikipedia.org/wiki/Variational_autoencoder) and Generative Adversarial Network [GAN](https://en.wikipedia.org/wiki/Generative_adversarial_network) models have achieved some success in the past. However, a good generative model should generate the data following the same distribution as the training data. There are some known flaws with the VAE and GAN models for synthetic data generation. Most prominently, the [mode collapse problem](https://developers.google.com/machine-learning/gan/problems) in the GAN model causes the generated data to miss some modes in the training data distribution. While the VAE has the difficulty of generating sharp data points due to the non-autoregressive loss.

Recently, [Transformer models](https://arxiv.org/abs/1706.03762) achieved huge success in the natural language processing domain.  The self-attention encoding and decoding architecture of the transformer model is proven to be accurate in modeling the data distribution and scalable to large datasets. [OpenAI’s GPT3](https://openai.com/blog/gpt-3-apps/) uses the decoder part of the transformer model and has 175B parameters. It has been widely used across varying categories and industries, from productivity and education to creativity and games. GPT3 turns out to be a superior generative model. In this tutorial, we are interested in exploring the ideas of applying the GPT model for synthetic data generation. 

Unlike typical neural networks, GPT models are usually large in size. It is challenging to fit a large GPT model into a single GPU. Luckily, [NeMo](https://github.com/NVIDIA/NeMo) is an open source tool that is capable of efficiently training very large (hundreds of billions of parameters) language models with both model and data parallelism.  It is easy to use too. In the following sections, we will show how to train a GPT model step by step to generate synthetic credit card data.


In [1]:
BRANCH = 'main'

# 1.Data Cleaning and Formatting

First let's import the necessary Python libraries for the tutorial.

In [2]:
import pandas as pd
from nemo.collections.common.tokenizers.column_coder import ColumnCodes
from omegaconf import OmegaConf
import pickle
from pandas.api.types import is_string_dtype
from nemo.collections.common.tokenizers.tabular_tokenizer import TabularTokenizer
import json
import wget
import requests
import json
import functools
from multiprocessing import Pool

/opt/conda/envs/nemo/lib/python3.9/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
BRANCH = 'main'
DATA_PATH='./Cohort_data'
TRANSACTIONS=DATA_PATH+'/HT_Pros_sdoh_trimed.csv'
#CHECKPOINTS='/chk_points'
CHECKPOINTS=DATA_PATH+'/checkpoints'
CC_OUTPUT_P=DATA_PATH+'/HT_Pros_coder.pickle'
CC_OUTPUT_J=DATA_PATH+'/HT_Pros_card.jn'
print (TRANSACTIONS, CHECKPOINTS, CC_OUTPUT_P, CC_OUTPUT_J)

./Cohort_data/HT_Pros_sdoh_trimed.csv ./Cohort_data/checkpoints ./Cohort_data/HT_Pros_coder.pickle ./Cohort_data/HT_Pros_card.jn


After decompressing the tar file,  we load the dataset into pandas dataframe and examine the top few rows.

In [4]:
df = pd.read_csv(TRANSACTIONS)
df.head()

,Unnamed: 0,patient_id,admityear,admitmonth,admitday,dischyear,dischmonth,dischday,chartyear,chartmonth,...,clm_pymt_amt_rm_9,snapspth12,hydroquinone,cera_30Alzheimer,vinylidene_chloride,flag_ratio_Hematocrit_normal,mp_limeng,acetonitrile,pentachlorophenol,drugs_cnt_uniq
0,0,0001694E47AE68F9,2009,3,26,2009,3,26,2009,3,...,0.0,0.591384,0.0001,0.0,0.001567,0.0,0.3,0.323546,0.0,0.0
1,1,0001694E47AE68F9,2009,8,7,2009,8,7,2009,8,...,0.0,0.591384,0.0001,0.0,0.001567,0.0,0.3,0.323546,0.0,0.0
2,2,0001694E47AE68F9,2009,8,20,2009,8,20,2009,8,...,0.0,0.591384,0.0001,0.0,0.001567,0.0,0.3,0.323546,0.0,0.0
3,3,0001694E47AE68F9,2009,9,13,2009,9,13,2009,9,...,0.0,0.591384,0.0001,0.0,0.001567,0.0,0.3,0.323546,0.0,0.0
4,4,0001694E47AE68F9,2009,11,24,2009,11,24,2009,11,...,0.0,0.591384,0.0001,0.0,0.001567,0.0,0.3,0.323546,0.0,0.0


In [5]:
del df['Unnamed: 0']

pd.options.display.max_rows = 4000column_type = df1.dtypesIn the following data cleaning step, we fix the `Amount` column that is not a number, convert the `Time` column into `hour` and `minutes` and replace the `,` character with space so we can use `,` as delimiter.

In [7]:
df['cera_30Hypertension'] = df['cera_30Hypertension'].astype("int")
df['cera30_sum'] = df['cera30_sum'].astype("int")
df['cera_30Alzheimer'] = df['cera_30Alzheimer'].astype("int")
df['cera_30AtrialFib'] = df['cera_30AtrialFib'].astype("int")

df['cera_30Covid'] = df['cera_30Covid'].astype("int") 
df['cera_30Depression']=df['cera_30Depression'].astype("int")
df['cera_30Diabetes']=df['cera_30Diabetes'].astype("int")
df['cera_30Alzheimer']=df['cera_30Alzheimer'].astype("int")
df['cera_30Pneumonia']=df['cera_30Pneumonia'].astype("int")

df['drugs_cnt_uniq'] = df['drugs_cnt_uniq'].astype('int')
df['resp_open'] = df['resp_open'].astype('int')
df['Trazodone'] = df['Trazodone'].astype('int')
                                                       

In [8]:
column_type = df.dtypes
column_type

patient_id                       object
admityear                         int64
admitmonth                        int64
admitday                          int64
dischyear                         int64
                                 ...   
flag_ratio_Hematocrit_normal    float64
mp_limeng                       float64
acetonitrile                    float64
pentachlorophenol               float64
drugs_cnt_uniq                    int64
Length: 110, dtype: object

In [9]:
column_type['cera_30Hypertension']

dtype('int64')

In [10]:
category_columns = ['location','county','st_abbr','state']
for col in category_columns:
    if is_string_dtype(df[col].dtype):
        df[col] = df[col].str.replace(',', ' ')

In [11]:
columns = column_type[1:]


In [12]:

float_columns = columns[(columns == "float64")].keys().tolist()

category_columns = columns[(columns != "int64") & (columns != "float64")].keys().tolist()

integer_columns = columns[(columns == "int64")].keys().tolist()

In [13]:
float_columns

['colvalue_agg_Glucose_normal',
 'mp_munit',
 'o_cresol',
 'acenaphthene',
 'spo2_amin',
 'benzoepyrene',
 'cum_clm_pymt_amt_rm_10',
 'lalowia1_20',
 'diethylene_glycol_monobutyl_ether',
 'carbon_tetrachloride',
 'dimethyl_sulfate',
 'fluorene',
 'laccess_seniors15',
 'b2_methylphenanthrene',
 'methyl_bromide',
 'ip_ddctbl_amt_rm_5',
 'benzobfluoranthene',
 'dera_30',
 'methyl_methacrylate',
 'cera_30Ischemic',
 'methylhydrazine',
 'm_crowd',
 'clm_pymt_amt_rm_10',
 'diethylene_glycol_monoethyl_ether',
 'dichloroethyl_ether',
 'recfac09',
 'acrolein',
 'm_disabl',
 'wght_mean_2',
 'acetaldehyde',
 'pc_dirsales07',
 'povrate15',
 'last_yr_cost',
 'carbon_disulfide',
 'colvalue_agg_Creatinine_abnormal',
 'propionaldehyde',
 'd4_nitrophenol',
 'clm_pymt_amt',
 'rpl_theme1',
 'clm_pymt_amt_rm_7',
 'epl_mobile',
 'n_n_dimethylaniline',
 'pct_fmrkt_sfmnp16',
 'propylene_dichloride',
 'diesel_pm',
 'colvalue_agg_Glucose_abnormal',
 'clm_pymt_amt_rm_9',
 'snapspth12',
 'hydroquinone',
 'vinyli

In [15]:
columns = columns.keys()

In [14]:
df = df[column_type.keys()]
df.head()

,patient_id,admityear,admitmonth,admitday,dischyear,dischmonth,dischday,chartyear,chartmonth,chartday,...,clm_pymt_amt_rm_9,snapspth12,hydroquinone,cera_30Alzheimer,vinylidene_chloride,flag_ratio_Hematocrit_normal,mp_limeng,acetonitrile,pentachlorophenol,drugs_cnt_uniq
0,0001694E47AE68F9,2009,3,26,2009,3,26,2009,3,26,...,0.0,0.591384,0.0001,0,0.001567,0.0,0.3,0.323546,0.0,0
1,0001694E47AE68F9,2009,8,7,2009,8,7,2009,8,7,...,0.0,0.591384,0.0001,0,0.001567,0.0,0.3,0.323546,0.0,0
2,0001694E47AE68F9,2009,8,20,2009,8,20,2009,8,20,...,0.0,0.591384,0.0001,0,0.001567,0.0,0.3,0.323546,0.0,0
3,0001694E47AE68F9,2009,9,13,2009,9,13,2009,9,13,...,0.0,0.591384,0.0001,0,0.001567,0.0,0.3,0.323546,0.0,0
4,0001694E47AE68F9,2009,11,24,2009,11,24,2009,11,24,...,0.0,0.591384,0.0001,0,0.001567,0.0,0.3,0.323546,0.0,0


There are 24M records with 12 fields. We need to convert the structured tabular data into a format that NeMo GPT can consume. The NeMo GPT uses training data in a loose json format, with one json containing a text sample per line. For example:
```json
{"text": "The quick brown fox"}
{"text": "jumps over the lazy dog"}
```
We can concat each fields of the transactions together like:
```json
{"text": "0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No\n0,0,2002,9,1,06:42,$38.48,Swipe,Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No..."}
...
```
In this way, the tabular data is converted into NLP text corpus. 

### NeMo GPT Workflow
```
                       +-----------+
      +--------------->|Tokenizer  |
      |                +-----+-----+
      |                      |
+-----+-----+          +-----v--------+      +---------+
|  Raw Data +--------->| Preprocess   +----->|Pretrain |
|           |          |              |      +-----+---+
+-----------+          +--------------+            |
                                                   |
                                           +-------v----------+
                                           |  Inference       |
                                           | (Text Generation)|
                                           +------------------+
```
As shown in the figure above, it illustrates the steps we are going to take in this tutorial. Having the raw text data ready, we first train a tokenizer to efficiently tokenize the text.  As NeMo only cares about sequences of tokens, a preprocess step is used to convert the raw text data into sequences of token index. It results in one binary file and an index file so data loading in the training step is more efficient.  The next step is the most time consuming pre-training step in which sequences of tokens are fed into the NeMo GPT to predict the next tokens. The prediction error is used as the loss to train the GPT weights. After training the GPT model, it can be put into an inference mode to generate synthetic data following the same distribution as the training dataset. 

# 2.Tokenizer training
It is natural to select the generic GPT BPE tokenizer to convert the text into tokens. However, there are a few problems of using this approach. When GPT BPE tokenizer splits the text into tokens, the number of tokens are usually not fixed for the same column at different rows, because the number is determined by the occurrence frequencies of the individual sub-tokens. This means the structure information is lost if we use the NLP tokenizer. Another problem with the NLP tokenizer is the long string consists of a large number of tokens, which is wasteful considering the NeMo GPT has limited capacity of modeling the sequences of tokens. For example the "Merchant Name" "3527213246127876953" need at least 7 tokens to code it ([35][27][213][246][127][876][953]). 

As shown in the [TabFormer paper](https://arxiv.org/abs/2011.01843), a good solution is to build a specialized tokenizer for the tabular data that considers the table structural information. The TabFormer uses a single token for each of the columns which can cause either accuracy loss if the number of tokens is small for the column, or weak generalization if the number of tokens is too large. We improve it by using multiple tokens to code the columns. For example, the floating number "134.09" can be tokenized into multiple integers.

The following code trains a special encoder/decoder used in the tokenizer. It uses `float` for the `Amount` column, `int` for the `zip` column and `category` for the rest of the columns. The trained encoder and decoder are saved into a file that can be loaded later. We choose to use 3 tokens to encode the floating and int numbers in this example.

In [ ]:

tab_structure = []
for c in columns:
    if c in float_columns:
        item = {
            "name": c,
            "code_type": "float",
            "args": {
                "code_len": 6,  # number of tokens used to code the column
                "base": 32,   # the positional base number. ie. it uses 32 tokens for one digit
                "fillall": True, # whether to use full base number for each token or derive it from the data.
                "hasnan": False, # can it handles nan or not
                "transform": "yeo-johnson" # can be ['yeo-johnson', 'quantile', 'robust'], check https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing 
            }
        }
    elif c in integer_columns:
        item = {
            "name": c,
            "code_type": "int",
            "args": {
                "code_len": 6,  # number of tokens used to code the column
                "base": 47,   # the positional base number. ie. it uses 32 tokens for one digit
                "fillall": True, # whether to use full base number for each token or derive it from the data.
                "hasnan": True, # can it handles nan or not
            }
        }
    else:
        item = {
            "name": c,
            "code_type": "category",
        }
    tab_structure.append(item)
print(tab_structure)
print(OmegaConf.to_yaml(tab_structure))
print(columns)

example_arrays = {}
for col in tab_structure:
    col_name = col['name']
    if col_name in category_columns:
        example_arrays[col_name] = [i.strip() for i in df[col_name].astype(str).unique()]
    else:
        example_arrays[col_name] = df[col_name].dropna().unique()
cc = ColumnCodes.get_column_codes(tab_structure, example_arrays)
print('each row uses', sum(cc.sizes)+ 1, 'tokens')
with open(CC_OUTPUT_P, 'wb') as handle:
    pickle.dump(cc, handle)


In [16]:
columns

Index(['admityear', 'admitmonth', 'admitday', 'dischyear', 'dischmonth',
       'dischday', 'chartyear', 'chartmonth', 'chartday',
       'cera_30Hypertension',
       ...
       'clm_pymt_amt_rm_9', 'snapspth12', 'hydroquinone', 'cera_30Alzheimer',
       'vinylidene_chloride', 'flag_ratio_Hematocrit_normal', 'mp_limeng',
       'acetonitrile', 'pentachlorophenol', 'drugs_cnt_uniq'],
      dtype='object', length=109)

Let's give it a try to play with encoder and decoder for "Amount" and "Merchant City" columns

In [19]:
float_str = '2009'
token_ids = cc.encode('admityear', float_str)
print('token ids for {} is {}'.format(float_str, token_ids))
amt_str = cc.decode('admityear', token_ids)
print('recovered Amt for {} is {}'.format(float_str, amt_str))

city_str = 'OHIO'
token_ids = cc.encode('state', city_str)
print('token ids for {} is {}'.format(city_str, token_ids))
amt_str = cc.decode('state', token_ids)
print('recovered City for {} is {}'.format(city_str, amt_str))

token ids for 2009 is [235, 188, 141, 94, 47, 0]
recovered Amt for 2009 is 2009
token ids for OHIO is [15635]
recovered City for OHIO is OHIO


Using 3 tokens for the float column `Amount`, this is a tiny loss of accuracy. If better accuracy is needed, more tokens can be used to train the tokenizer. For the categorical column "Merchant City", the string "Monterey Park" only needs one token to encode it. Once we have the encoder and decoder ready, the tokenizer is easy to implement. We can consider the tabular data structure information to encode each of the columns. The decoding simply counts the number of tokens to infer the tabular structure since the number of the tokens are fixed for each of the columns.

There is one more thing that we need to take special care of before generating the loose json file for NeMo. `<|endoftext|>` is a special token that NeMo recognizes to indicate the beginning and end of the document. The attention mask will stop at the boundary of `<|endoftext|>` token so no attention can be applied across it. To model the temporal information in the time series, we want to make sure the `<|endoftext|>` is added between the continuous sections. For example, in this credit card dataset, there are 2000 users. Each user's transactions are one long time series sequence. The `<|endoftext|>` is added at the end of the long sequences. In this way, NeMo applies attention to learn the temporal correlation in the transactions for the user but not across users. 

We have provided the Python code to convert the CSV file into the loose json format.

In [59]:
delimiter = ','
eod_str = '<|endoftext|>'
int_nan = '<NA>'


def get_docs_from_df(one_group):
    user_df = one_group[1]
    total = []
    # sort the rows by time, so the model can learn temporal info
    sub_df = user_df.sort_values(['admityear', 'admitmonth', 'admitday','dischyear','dischmonth','dischday','chartyear','chartmonth','chartday'])[columns]
    start_rows = range(0, doc_augements)
    full_msgs = []
    for start in start_rows:
        full_msgs = []
        doc_df = sub_df.iloc[start:]
        count = 0
        df_size = len(doc_df)       
        for row in doc_df.iterrows():
            count += 1
            items = row[1].values
            str_items = [str(items[i]).replace(int_nan, 'nan') for i in range(len(items))]
            if count == df_size:
                # append eod at the end of the doc
                full_msgs.append(delimiter.join(str_items)+eod_str)
            else:
                full_msgs.append(delimiter.join(str_items))
        # use end of line to seperate rows
        text = '\n'.join(full_msgs)
        text_doc = {'text': text}
        doc = json.dumps(text_doc)+'\n'
        total.append(doc)
    return total


def gen_one_doc(user_group, n_cores):
    udfs = list(user_group)
    pool = Pool(n_cores)
    docs = pool.map(get_docs_from_df, udfs)
    pool.close()
    pool.join()
    return functools.reduce(lambda a, b: a + b, docs)

# number of document augmentation
doc_augements = 2

user_group = df.groupby('patient_id')

with open(CC_OUTPUT_J, 'w') as f:
    docs = gen_one_doc(user_group, 16)
    for doc in docs:
        f.write(doc)

In [60]:
user_group.mean()

,admityear,admitmonth,admitday,dischyear,dischmonth,dischday,chartyear,chartmonth,chartday,cera_30Hypertension,...,clm_pymt_amt_rm_9,snapspth12,hydroquinone,cera_30Alzheimer,vinylidene_chloride,flag_ratio_Hematocrit_normal,mp_limeng,acetonitrile,pentachlorophenol,drugs_cnt_uniq
patient_id,,,,,,,,,,,,,,,,,,,,,
0001694E47AE68F9,2009.333333,6.888889,17.444444,2009.333333,6.888889,17.444444,2009.333333,6.888889,17.444444,0.000000,...,0.000000,0.591384,0.000100,0.000000,0.001567,0.0,0.30,0.323546,0.000000,0.0
0002056B40CEE448,2009.272727,5.818182,13.181818,2009.272727,5.818182,13.863636,2009.272727,5.818182,13.181818,0.136364,...,0.000000,0.696801,0.000153,0.045455,0.000193,0.0,0.30,2.941432,0.000000,0.0
000240D599ED789C,2009.363636,4.363636,13.727273,2009.363636,4.454545,13.636364,2009.363636,4.363636,13.727273,0.090909,...,3381.212121,0.621625,0.003974,0.090909,0.000688,0.0,0.30,0.157387,0.000001,0.0
0004FB2ED3823AA8,2009.538462,6.307692,15.846154,2009.576923,5.961538,14.346154,2009.538462,6.307692,15.846154,0.153846,...,557.136752,0.820179,0.008379,0.000000,0.003886,0.0,0.20,1.762268,0.000000,0.0
0007B80E222CD8B0,2009.583333,5.083333,16.416667,2009.583333,5.083333,17.416667,2009.583333,5.083333,16.416667,0.166667,...,0.000000,0.864417,0.020482,0.000000,0.020914,0.0,0.10,0.051159,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D3FDEE12EE74164F,2009.000000,2.000000,8.666667,2009.000000,2.000000,8.666667,2009.000000,2.000000,8.666667,0.000000,...,0.000000,0.853956,0.007855,0.000000,0.004064,0.0,0.30,0.057389,0.000041,0.0
D3FE36C6F7419237,2009.000000,3.000000,16.500000,2009.000000,3.000000,16.500000,2009.000000,3.000000,16.500000,0.000000,...,0.000000,0.853956,0.007855,0.000000,0.004064,0.0,0.30,0.057389,0.000041,0.0
D401ED967E8DD980,2009.000000,5.000000,7.500000,2009.000000,5.000000,7.500000,2009.000000,5.000000,7.500000,0.000000,...,0.000000,0.853956,0.007855,0.000000,0.004064,0.0,0.30,0.057389,0.000041,0.0


After running the above cell, it converts the CSV file to the loose json format that the NeMo Megatron can use. It generates one document for each user. Each document ends with the `<|endoftext|>` token. We augment the data by shifting it by one transaction so the relative position of the input credit card transaction data is slightly different. 

Having the loose JSON file ready, we can test the customized Tabular Tokenizer.

In [20]:
tokenizer = TabularTokenizer(CC_OUTPUT_P, delimiter=',')

with open(CC_OUTPUT_J, 'r') as f:
    for line in f:
        break

text = json.loads(line)['text']
r = tokenizer.text_to_tokens(text)
ids = tokenizer.tokens_to_ids(r)
tex = tokenizer.ids_to_tokens(ids)
decoded_text = tokenizer.ids_to_text(ids)
show_num_lines = 5
print('raw input text\n', '\n'.join(text.split('\n')[:show_num_lines]))
print('tokens\n', r[:show_num_lines*len(tokenizer.code_column.columns)])
print('token ids\n', ids[:show_num_lines*(sum(tokenizer.code_column.sizes)+1)])
print('decoded tokens\n', tex[:show_num_lines*(sum(tokenizer.code_column.sizes)+1)])
print('decoded text\n', '\n'.join(decoded_text.split('\n')[:show_num_lines]))
print(len(r[:show_num_lines*len(tokenizer.code_column.columns)]))

KeyError: 'Oktibbeha County_ Mississippi'

In [21]:
tokenizer = TabularTokenizer(CC_OUTPUT_P, delimiter=',')

with open(CC_OUTPUT_J, 'r') as f:
    for line in f:
        break
text = '2010,4,6,2010,4,6,2010,4,6,0,1,1,1,0,0,0,0.00000000,Delaware County  Pennsylvania,0.39999999,0.00676585,Delaware,1,0,1,0,0.00857885,0,0,0.00000000,0.00331584,0.00000000,0,0,0,14276.23142521,0,0,8.66480600,0.00539408,0,0,0.00001180,0.01655906,13472.32828466,0,PA,0.00116019,0,0.00026271,PENNSYLVANIA,0.00000000,0.00192493,0.00000000,0.22055985,0.00000000,0.00000000,303.09999698,0,0,0.00000000,0.00000000,0,0.00000000,0,83.99999783,0.51579630,1487.99998394,0,0.00000000,0,3.30377683,0.08830164,0,0,0,10.39999987,1,0,0.00000000,0.01500722,0.00000000,0.28498782,0.01480469,0,0.00000000,0.20529999,0.00000000,0.01079999,0,0,0,0,0.00290252,83.33333270,0,0.00034082,9.82300603,0,0.00000000,0.00000000,0.69298648,0.01072868,0,0.00381294,0.00000000,0.19999999,0.00157357,0.00000090,0\n<|endoftext|>'
#text = json.loads(line)['text']
r = tokenizer.text_to_tokens(text)
ids = tokenizer.tokens_to_ids(r)
tex = tokenizer.ids_to_tokens(ids)
decoded_text = tokenizer.ids_to_text(ids)
show_num_lines = 5
print('raw input text\n', '\n'.join(text.split('\n')[:show_num_lines]))
print('tokens\n', r[:show_num_lines*len(tokenizer.code_column.columns)])
print('token ids\n', ids[:show_num_lines*(sum(tokenizer.code_column.sizes)+1)])
print('decoded tokens\n', tex[:show_num_lines*(sum(tokenizer.code_column.sizes)+1)])
print('decoded text\n', '\n'.join(decoded_text.split('\n')[:show_num_lines]))
print(len(r[:show_num_lines*len(tokenizer.code_column.columns)]))

raw input text
 2010,4,6,2010,4,6,2010,4,6,0,1,1,1,0,0,0,0.00000000,Delaware County  Pennsylvania,0.39999999,0.00676585,Delaware,1,0,1,0,0.00857885,0,0,0.00000000,0.00331584,0.00000000,0,0,0,14276.23142521,0,0,8.66480600,0.00539408,0,0,0.00001180,0.01655906,13472.32828466,0,PA,0.00116019,0,0.00026271,PENNSYLVANIA,0.00000000,0.00192493,0.00000000,0.22055985,0.00000000,0.00000000,303.09999698,0,0,0.00000000,0.00000000,0,0.00000000,0,83.99999783,0.51579630,1487.99998394,0,0.00000000,0,3.30377683,0.08830164,0,0,0,10.39999987,1,0,0.00000000,0.01500722,0.00000000,0.28498782,0.01480469,0,0.00000000,0.20529999,0.00000000,0.01079999,0,0,0,0,0.00290252,83.33333270,0,0.00034082,9.82300603,0,0.00000000,0.00000000,0.69298648,0.01072868,0,0.00381294,0.00000000,0.19999999,0.00157357,0.00000090,0
<|endoftext|>
tokens
 ['2010', '4', '6', '2010', '4', '6', '2010', '4', '6', '0', '1', '1', '1', '0', '0', '0', '0.00000000', 'Delaware County  Pennsylvania', '0.39999999', '0.00676585', 'Delaware', '1', '0',

In [ ]:
['<|endoftext|>2010,4,6,2010,4,6,2010,4,6,0,1,1,1,0,0,0,0.00000000,Delaware County  Pennsylvania,0.39999999,0.00676585,Delaware,1,0,1,0,0.00857885,0,0,0.00000000,0.00331584,0.00000000,0,0,0,14276.23142521,0,0,8.66480600,0.00539408,0,0,0.00001180,0.01655906,13472.32828466,0,PA,0.00116019,0,0.00026271,PENNSYLVANIA,0.00000000,0.00192493,0.00000000,0.22055985,0.00000000,0.00000000,303.09999698,0,0,0.00000000,0.00000000,0,0.00000000,0,83.99999783,0.51579630,1487.99998394,0,0.00000000,0,3.30377683,0.08830164,0,0,0,10.39999987,1,0,0.00000000,0.01500722,0.00000000,0.28498782,0.01480469,0,0.00000000,0.20529999,0.00000000,0.01079999,0,0,0,0,0.00290252,83.33333270,0,0.00034082,9.82300603,0,0.00000000,0.00000000,0.69298648,0.01072868,0,0.00381294,0.00000000,0.19999999,0.00157357,0.00000090,0\n']

The TabularTokenizer understands the Table structure so it can convert back and forth between the tabular data text and the token ids.


# 3.Model configuration


We define the following constants to configure the GPT model 

In [ ]:
# NUM_LAYERS = 4
# NUM_GPUS = 1
# HIDDEN_SIZE = 1024
# NUM_ATTENTION_HEADS = 16
# SEQ_LENGTH = 1024
# TENSOR_MP_SIZE = 1
# PIPELINE_MP_SIZE = 1

In [26]:
NUM_LAYERS = 2
NUM_GPUS = 1
HIDDEN_SIZE = 512
NUM_ATTENTION_HEADS = 8
SEQ_LENGTH = 512
TENSOR_MP_SIZE = 1
PIPELINE_MP_SIZE = 1

# 4.Preprocess

The loose json above is processed into a binary format for training. To convert the json into mmap, cached index file, we use the `preprocess_data_for_megatron.py` script to prepare the data.


Download the python file and run the preprocess. Note we use `30` CPU workers for the preprocessing step. Please adjust it according to your compute environment.

In [27]:
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/nlp_language_modeling/preprocess_data_for_megatron.py')

'preprocess_data_for_megatron (1).py'

In [28]:
!python preprocess_data_for_megatron.py \
    --input={CC_OUTPUT_J} \
    --json-keys=text \
    --tokenizer-library=tabular \
    --vocab-file={CC_OUTPUT_P} \
    --tokenizer-type=Tabular \
    --output-prefix=tabular_data \
    --delimiter=, \
    --workers=1


Traceback (most recent call last):
  File "preprocess_data_for_megatron.py", line 93, in <module>
    import ftfy
ModuleNotFoundError: No module named 'ftfy'


In [ ]:
!cp /content/tabular_data_text_document.idx /content/drive/MyDrive/BioMegatron/Synthetic_Data

In [19]:
!ls $DATA_PATH/tabular*.*

/content/drive/MyDrive/BioMegatron/Synthetic_Data/tabular_data_text_document.bin
/content/drive/MyDrive/BioMegatron/Synthetic_Data/tabular_data_text_document.idx


After running the script, two new files are generated in the current directory: `tabular_data_text_document.bin` and `tabular_data_text_document.idx`. The `bin` file is the `mmap` binary file and the `idx` is the index file to look up the sentences in the binary file. They will be used for the following pre-training step.

# 5.Pretrain

We are ready to pretrain the NeMo GPT model. As large models can be quite difficult to train due to memory constraints, NeMo makes it possible by using both Tensor model parallel and Pipeline model parallelism that enables training transformer models with billions of parameters. On top of the model parallelism, we can apply the popular data parallelism to the training to fully utilize all the GPUs in the cluster. In this [paper](https://arxiv.org/pdf/2104.04473.pdf), it provides a few tips about how to setup the model parallelism and data parallelism optimally:

1. When considering different forms of model parallelism, tensor model parallelism should generally be used up to degree 𝑔 when using 𝑔-GPU servers, and then pipeline model parallelism can be used to scale up to larger models across server.
2. When using data and model parallelism, a total model-parallel size of 𝑀 = 𝑡 · 𝑝 should be used so that the model’s parameters and intermediate metadata fit in GPU memory; data parallelism can be used to scale up training to more GPUs.
3. The optimal microbatch size 𝑏 depends on the throughput and memory footprint characteristics of the model, as well as the pipeline depth 𝑝, data-parallel size 𝑑, and batch size 𝐵.

In this tutorial, we only concern with training a model that fits into a single GPU. we set the tensor model parallel and pipeline model parallelism parameter to 1. Feel free to adjust it to train a larger model according to the available compute resources. As we know from OpenAI's [scaling law of natural language](https://arxiv.org/abs/2001.08361), the larger the model, the better the performance is.

Download and run the pretraining task script below. Note, this is the most time consuming step. It takes days for the task to converge depending on the computation environment, model size.

In [6]:
!mkdir -p conf
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/nlp/language_modeling/megatron_gpt_pretraining.py')
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/nlp/language_modeling/conf/megatron_gpt_config.yaml', out='conf')

'conf/megatron_gpt_config.yaml'

In [21]:
!python megatron_gpt_pretraining.py \
        trainer.devices={NUM_GPUS} \
        trainer.accelerator=gpu \
        trainer.log_every_n_steps=100 \
        trainer.val_check_interval=500 \
        trainer.accumulate_grad_batches=1 \
        trainer.max_steps=10000 \
        trainer.precision=16 \
        trainer.gradient_clip_val=1.0 \
        exp_manager.exp_dir=gpt_creditcard_results \
        model.tensor_model_parallel_size={TENSOR_MP_SIZE} \
        model.pipeline_model_parallel_size={PIPELINE_MP_SIZE} \
        model.optim.name=fused_adam \
        model.optim.lr=2e-4 \
        model.optim.sched.warmup_steps=2 \
        model.optim.sched.constant_steps=2 \
        model.optim.sched.min_lr=8e-5 \
        model.max_position_embeddings={SEQ_LENGTH} \
        model.encoder_seq_length={SEQ_LENGTH} \
        model.data.seq_length={SEQ_LENGTH} \
        model.tokenizer.type=Tabular \
        model.tokenizer.library=tabular \
        model.tokenizer.vocab_file={CC_OUTPUT_P} \
        model.tokenizer.delimiter=\',\' \
        model.data.eod_mask_loss=True \
        model.data.splits_string=\'3800,198,2\' \
        model.num_layers={NUM_LAYERS} \
        model.hidden_size={HIDDEN_SIZE} \
        model.num_attention_heads={NUM_ATTENTION_HEADS} \
        model.activations_checkpoint_method=\'block\' \
        model.activations_checkpoint_num_layers=1 \
        model.data.data_prefix=[tabular_data_text_document]


/usr/local/lib/python3.7/dist-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
[NeMo W 2022-07-26 00:19:17 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
      _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
    
[NeMo W 2022-07-26 00:19:27 experimental:28] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-07-26 00:19:27 experimental:28] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.Thutmo

The pretraining will save the checkpoint files periodically at the location specified by `exp_manager.exp_dir`. In this case, it creates a directory called `gpt_creditcard_results`. The number `trainer.val_check_interval` controls the frequency of validation evaluation. We use 500 in this example. The trainer will preserve the checkpoint files which have the top `k` best validation error, by default, `k=10`. 
While the training job is running, we can use the `tensorboard` to monitor the training. For example, you can run this command and check the learning curves at browser.
```bash
--logdir gpt_creditcard_results/ --bind_all 0.0.0.0
```

Once the training converges, we take a checkpoint file which has the smallest validation error.

# 6.Convert checkpoint to nemo file

NeMo library package the checkpoint file and other model artifacts (like model config file and vocab file) into a single `.nemo` file. It makes the process of loading GPT model and running inference easier. 

Let's download the script and convert the checkpoint file into `.nemo` file. Make sure to change the checkpoint file name to the correct name. You can find them in the `gpt_creditcard_results/megatron_gpt/checkpoints/ ` directory.

In [53]:
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/megatron_ckpt_to_nemo.py')

'megatron_ckpt_to_nemo.py'

In [ ]:
CHECKPONT_FILE_NAME = 'megatron_gpt--val_loss=1.17-step=10047-consumed_samples=80376.0-last.ckpt' # change it to your checkpoint file name
!python -m torch.distributed.launch --nproc_per_node=1 megatron_ckpt_to_nemo.py \
    --checkpoint_folder=cohort_results/megatron_gpt/checkpoints/ \
    --checkpoint_name='megatron_gpt--val_loss=0.09-step=100000-consumed_samples=800000.0.ckpt' \
    --nemo_file_path=Cohort_Modify_tabular.nemo \
    --tensor_model_parallel_size=1 \
    --pipeline_model_parallel_size=1 \
    --gpus_per_node=1 \
    --model_type=gpt

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
[NeMo W 2022-07-07 22:46:12 experimental:28] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-07-07 22:46:12 experimental:28] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2022-

After running this script, it creates a `tabular.nemo` file which can be used to generate synthetic credit card transactions. 

# 7.Generate synthetic credit card transactions

We come to the last step of this tutorial - generate synthetic credit card transactions!

First let's load the trained model weights, put the NeMo GPT in inference mode and start the text generation server. All of these can be done by running the `megatron_gpt_eval.py` script. 

Let's download the script and configuration file.

In [54]:
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/megatron_gpt_eval.py')
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/megatron_gpt_inference.yaml', out='conf')

'conf/megatron_gpt_inference.yaml'

Open another Jupyter notebook or terminal, and run the following in a cell. 
```python
!python megatron_gpt_eval.py \
    gpt_model_file=tabular.nemo \
    prompts=[\'\',\'\'] \
    server=True
```
The text generation server accepts REST API request to send the generated text in the response. Let's use the following Python code to generate some transactions.

In [26]:
with open('Cohort_data/HT_Pros_coder.pickle', 'rb') as handle:
        cc: ColumnCodes = pickle.load(handle)

token_per_rows = sum(cc.sizes) + 1

batch_size = 1
port_num = 5555
num_of_rows = 1
headers = {"Content-Type": "application/json"}


def request_data(data):
    
    resp = requests.put('http://localhost:{}/generate'.format(port_num),
                        data=json.dumps(data), headers=headers)
    print(data)
    sentences = resp.json()['sentences']
    return sentences


# generate the initial transactions 
data = {
    "sentences": [""] * batch_size,
    "tokens_to_generate": num_of_rows * token_per_rows,
    "temperature": 1.0,
    "add_BOS": True
}

sentences = request_data(data)
print('sen',sentences)
for i in range(batch_size):
    s = sentences[i]
    print(s)
    

{'sentences': [''], 'tokens_to_generate': 635, 'temperature': 1.0, 'add_BOS': True}
sen ['<|endoftext|>2009,7,3,2009,7,3,2009,7,3,1,1,1,1,0,0,0,0.00000000,Hudson County_ New Jersey,0.49999999,0.00948468,Hudson,0,0,1,0,0.00662348,0,0,0.00000000,0.00118016,0.00000000,0,0,0,997.85564644,0,0,4.83223659,0.01000000,0,0,0.00606622,0.01194711,382.14268084,0,NJ,0.00016825,0,0.04028550,NEW JERSEY,0.00000000,0.00104126,0.00000000,0.05590414,0.00000000,0.04284015,919.89999601,0,0,0.00000000,0.00000000,0,0.00000000,0,59.99999915,0.19854241,1513.99998066,0,0.00000000,0,1.63537937,0.00000000,1,0,0,17.69999975,0,0,0.00000000,0.03637620,0.00000000,0.23161238,0.02055448,0,-0.00000000,0.52120000,0.00000000,0.00639999,0,0,0,0,0.00344984,74.99999975,0,0.00012321,16.44639380,0,0.00000000,0.00000000,1.12716195,0.00155581,0,0.00453193,0.00000000,0.40000000,0.00369626,0.00000000,0\n']
<|endoftext|>2009,7,3,2009,7,3,2009,7,3,1,1,1,1,0,0,0,0.00000000,Hudson County_ New Jersey,0.49999999,0.00948468,Hudson,0,0,1,0

In [53]:
df_out['drugs_cnt_uniq'] =0

In [54]:
df_out['drugs_cnt_uniq']

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: drugs_cnt_uniq, Length: 1000, dtype: int64

In [57]:
df_out.to_csv('synthetic_csv.csv')

The above code only generate `num_of_rows` of transactions starting from the beginning. If longer sequence is needed, we can run the inference conditioned on the past transactions in a sliding window fashion. For example, first it generates [A, B, C, D, E] transactions conditioned on `<|endoftext|>` token. Then, it conditions on [D, E] and generates [D, E, F, G, H]. Once the long sequence comes to the end indicated by `<|endoftext|>`, it will keep generating new transactions for a different user. For example, after generating [X, Y, Z, <|endoftext|>] in the last run, it will generate [Z, <|endoftext|>, A', B', C'] in the next time, where A', B', C' are transactions for a different user and they are not depending on the other users transaction Z.  

The following code implements the idea above and can be used to generate massive number of transactions in long sequences. The `history_rows` specifies the number of rows of transactions used as conditional context and `num_of_rows` specify the number of rows of transactions to be generated at a time. It saves all the synthetic credit card transactions in the text files whose file names starts with prefix `synthetic`. You can adjust `number_of_blocks`, `batch_size` parameters to generate more transactions.  

In [44]:
with open('Cohort_data/HT_Pros_coder.pickle', 'rb') as handle:
        cc: ColumnCodes = pickle.load(handle)
print(sum(cc.sizes))
batch_size = 1
num_of_rows = 50
token_per_rows = sum(cc.sizes)+1
history_rows = 40
num_of_blocks = 10

port_num = 5555
headers = {"Content-Type": "application/json"}

prefix_name = 'synthetic'
files = []


for i in range(batch_size):
    files.append(open("{}_{}.txt".format(prefix_name, i), 'w'))


def request_data(data):
    resp = requests.put('http://localhost:{}/generate'.format(port_num),
                        data=json.dumps(data), headers=headers)
    
    sentences = resp.json()["sentences"]
    print(len(sentences))
    # for i in range(len(sentences)):
    #     if sentences[i][:13] == '<|endoftext|>':
    #         sentences[i] = sentences[i][13:]
    return sentences


def get_condition_text(sentences, history_rows):
    condition_text = ['\n'.join([ss for ss in s.split(
        '\n')[-(history_rows+1):]]) for s in sentences]
    # condition_text = ['<|endoftext|>'.join([ss for ss in s.split(
    #      '<|endoftext|>')[-(history_rows+1):]]) for s in sentences]
    return condition_text


def get_extra_text(sentences, history_rows):
    extra_text = ['\n'.join([ss for ss in s.split(
        '\n')[history_rows:]]) for s in sentences]
    return extra_text

# generate the inital transactions 
data = {
    "sentences": [""] * batch_size,
    "tokens_to_generate": num_of_rows * token_per_rows,
    "temperature": 1,
    "add_BOS": True
}

sentences = request_data(data)

print(sentences)
for i in range(batch_size):
    s = sentences[i]
    files[i].write(s.replace('<|endoftext|>', '\n'))
print('Original')
# generate the transactions conditioned on the previous ones
for block in range(num_of_blocks):
    print("block id: {}".format(block))
    condition_text = get_condition_text(sentences, history_rows)
    data = {
        "sentences": condition_text,
        "tokens_to_generate": num_of_rows * (token_per_rows),
        "temperature": 1,
        "add_BOS": False
    }
    #print(data)
    sentences = request_data(data)
    
    extra_text = get_extra_text(sentences, history_rows)
    print('extra_text',extra_text)
    for i in range(batch_size):
        s = extra_text[i]
        files[i].write(s.replace('<|endoftext|>', '\n---------------\n'))
        files[i].flush()



for i in range(batch_size):
    files[i].close()
    print('done')

634
1
['<|endoftext|>2009,8,5,2009,8,5,2009,8,5,0,0,0,1,0,0,0,0.00000000,DuPage County  Illinois,0.39999999,0.01146859,DuPage,0,0,0,0,0.02411849,0,0,0.00000000,0.00320100,0.00000000,0,0,0,33142.58795002,0,0,38.51639122,0.01263096,0,0,0.00001790,0.04483753,23535.01899925,0,IL,0.00000000,0,0.00027097,ILLINOIS,0.00000000,0.00514994,0.00000000,0.18736810,0.00000000,0.00000000,505.19999700,0,0,0.00000000,0.20474800,0,0.00000000,0,112.99999799,0.70947094,1564.99998712,0,0.00000000,0,6.44739868,0.14987446,0,0,0,7.09999984,0,0,0.00000000,0.02450772,0.00000000,0.62586532,0.03907797,0,0.00000000,0.08629999,0.00000000,0.00639999,20,0,0,0,0.00724985,5.88235274,0,0.00064839,34.50950248,0,0.00000000,0.00000000,0.43256712,0.00094952,0,0.00478976,0.00000000,0.19999999,0.21915005,0.00000430,0\n2009,9,20,2009,9,20,2009,9,20,0,0,0,1,0,0,0,0.00000000,DuPage County  Illinois,0.39999999,0.01146859,DuPage,0,0,0,0,0.02411849,0,0,0.00000000,0.00320100,0.00000000,0,0,0,33142.58795002,0,0,38.51639122,0.01263096,

KeyError: 'sentences'

In [87]:
# check the generated creditcard transaction files
!ls synthetic*.txt

synthetic_0.txt  synthetic_1.txt  synthetic_2.txt  synthetic_3.txt


That's it! In this tutorial, you have learned how to train a NeMo GPT model to generate synthetic data. Go ahead and apply it for your own data.

# Evaluation
Here is the coarse grained evaluation, to comput how many rows are copied from the real data

In [44]:
with open('Cohort_data/HT_Pros_coder.pickle', 'rb') as handle:
        cc: ColumnCodes = pickle.load(handle)

        
        
OUTPUT_FILE_NAME = 'synthetic_csv.csv'
NUM_OF_ROWS = 1000

token_per_rows = sum(cc.sizes) + 1
#token_per_rows = 545
batch_size = 1
port_num = 5555
num_of_rows = 1
headers = {"Content-Type": "application/json"}


def request_data(data):
    
    resp = requests.put('http://localhost:{}/generate'.format(port_num),
                        data=json.dumps(data), headers=headers)
    
    sentences = resp.json()['sentences']
    return sentences


# generate the initial transactions 
data = {
    "sentences": [""] * batch_size,
    "tokens_to_generate": num_of_rows * token_per_rows,
    "temperature": 1.0,
    "add_BOS": True
}
df_out = pd.DataFrame(columns=columns.tolist())
for _ in range(NUM_OF_ROWS):
    sentences = request_data(data)
    for i in range(batch_size):
        if sentences[i][:13] == '<|endoftext|>':
                sentences[i] = sentences[i][13:]
        s = sentences[i][:-2].split(',')
        # print(s)
        df_out.loc[len(df_out.index)] = s
df_out.to_csv(OUTPUT_FILE_NAME)
df_out.head()

In [4]:
df_out = pd.read_csv('synthetic_csv.csv')

In [ ]:
df_out['admittime']=df_out['admityear']+'_'+df_out['admitmonth']+'_'+df_out['admitday']


df_out['dischtime']=df_out['dischyear']+'_'+df_out['dischmonth']+'_'+df_out['dischday']


df_out['charttime']=df_out['chartyear']+'_'+df_out['chartmonth'].str+'_'+df_out['chartday']


In [ ]:
columns_titles = ["B","A"]

In [1]:
total = pd.concat([df, df_out])
copies = len(total) - len(total.drop_duplicates()) - (len(df) - len(df.drop_duplicates())) - (len(df_out) - len(df_out.drop_duplicates()))
print(copies)


NameError: name 'pd' is not defined

In [59]:
100*(len(df_out) - len(df_out.drop_duplicates()))/len(df_out)

0.0